# Calcium Imaging Motion Correction Using SIMA

author: Zhe Charles Zhou (UW NAPE Center)

Loads in raw calcium imaging movies and performs motion correction using the SIMA package. The algorithm used by SIMA is based on a Hidden Markov Model to estimate x-y shifts of each frame's lines.

Specific SIMA code and detailed methods can be found in: 

https://github.com/losonczylab/sima

https://www.frontiersin.org/articles/10.3389/fninf.2014.00080/full

pre-req input data:

- raw calcium imaging videos in tif or h5 format

In [1]:
#import sima and other dependents
import sima
import sima.motion
from sima.motion import HiddenMarkov2D
import numpy as np
import os
import pickle
import h5py

In [2]:
"""

    Input arguments:
    
        fname    : string
            Raw video file name
            
        dir      : string
            Path of the root directory where the raw data resides
        
        max_disp : list of two elements
            The maximum allowed displacement magnitudes in pixels in [y,x]. 
            Both values must be ints (integers).
    
        tif_h5   : int
            set as 0 if the raw input data is a tiff, 1 for h5
            
    Output:
    
        _sima_mc.h5 file
            Motion corrected raw data in h5 format. 
            Dimensions of data are (frame, plane, row-y, column-x, channel). 
        
        sima_mc_std.tif file
            This code will output several projections (images collapsed across the frame/time dimension).
            By default, this will save standard dev, mean, and kurtosis projection images.

        displacement.pkl file
            SIMA provides the calculated x and y shifts for each frame's lines made during the motion correction.
"""

# USER DEFINE the following four variables; see comments above for details:
fname = 'Calca-hm3-mc_0002' 
fdir = 'C:\\2pData\\Jane\\' 
max_disp = [10, 10] 
tif_h5 = 0 

In [3]:
# get full path of file and load via sima

if tif_h5 == 0:
    
    # splices file and file directory into a single path for loading
    datafile = os.path.join(fdir, '%s.tif'%fname)
    # sequence: object that contains record of whole dataset; data not stored into memory all at once
    sequences = [sima.Sequence.create('TIFF', datafile)] 
    
elif tif_h5 == 1:
    
    datafile = os.path.join(fdir, '%s.h5'%fname)
    sequences = [sima.Sequence.create('HDF5', datafile, 'tyx')]
    
display(datafile)

'C:\\2pData\\Jane\\Calca-hm3-mc_0002.tif'

(1000, 1L, 512L, 512L, 1L)

In [5]:
# define motion correction method
# n_processes can only handle =1! Bug in their code where >1 runs into an error
# max_displacement: The maximum allowed displacement magnitudes in pixels in [y,x]
mc_approach = sima.motion.HiddenMarkov2D(granularity='row', max_displacement=max_disp, n_processes = 1, verbose=True)

## Apply motion correction

In [10]:
%%time
# apply motion correction to data
dataset = mc_approach.correct(sequences, os.path.join(fdir, fname + '_mc.sima'), channel_names=['GCaMP'])
# dataset dimensions are frame, plane, row(y), column (x), channel

Estimating model parameters.
Estimating displacements for cycle  0
Wall time: 3min 35s


In [11]:
# Save motion corrected files : h5 as well as projections
dataset.export_frames([os.path.join(fdir, fname + '_sima_mc.h5')], fmt='HDF5') # can change fmt to TIFF16 as well
dataset.export_averages([os.path.join(fdir, fname + '_sima_mc_std.tif')], projection_type='std')
dataset.export_averages([os.path.join(fdir, fname + '_sima_mc_mean.tif')], projection_type='average')
dataset.export_averages([os.path.join(fdir, fname + '_sima_mc_kurt.tif')], projection_type='kurtosis')

### Calculate motion displacement

In [ ]:
%%time
# show motion displacements after motion correction
mcDisp_approach = sima.motion.HiddenMarkov2D(granularity='row', max_displacement=max_disp, n_processes = 1, verbose=True)
displacements = mcDisp_approach.estimate(dataset)

In [ ]:
# save the resulting displacement file
# only useful if you want to see the values of displacement calculated by SIMA to perform the motion correction
displacement_file = open( os.path.join(fdir, fname + '_mc.sima/displacement.pkl'), "wb" )
pickle.dump( displacements, displacement_file )
displacement_file.close()

In [ ]:
# process and save np array of composite displacement
data_dims = displacements[0].shape
disp_np = np.squeeze( np.array(displacements[0]) )
disp_meanpix = np.mean( disp_np, axis=1 ) # avg across lines (y axis)

sima_disp = np.sqrt( np.square(disp_meanpix[:,0]) + np.square(disp_meanpix[:,1]) ) # calculate composite x + y offsets
np.save( os.path.join(fdir, 'displacements\\displacements_sima.npy'), sima_disp)